In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from IPython.display import display

In [20]:
#import csv
#use full location including raw.githubusercontent
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv", sep = ",")
#print(df.head())
print(df.head().T) # transposes, making rows columns


                                 0             1               2  \
customerID              7590-VHVEG    5575-GNVDE      3668-QPYBK   
gender                      Female          Male            Male   
SeniorCitizen                    0             0               0   
Partner                        Yes            No              No   
Dependents                      No            No              No   
tenure                           1            34               2   
PhoneService                    No           Yes             Yes   
MultipleLines     No phone service            No              No   
InternetService                DSL           DSL             DSL   
OnlineSecurity                  No           Yes             Yes   
OnlineBackup                   Yes            No             Yes   
DeviceProtection                No           Yes              No   
TechSupport                     No            No              No   
StreamingTV                     No            No

In [21]:
# cleaning
#df.dtypes
df.columns = df.columns.str.lower().str.replace(' ', '_') #making columns lower case and replacing space with underscore
#identify string columns
string_cols = df.select_dtypes(include='object').columns.tolist()
categorical_columns = string_cols
#print(string_cols)

for c in categorical_columns:
        df[c] = df[c].str.lower().str.replace(' ', '_')

# checking for null values in columns
# df[tc.isnull()][['customerid', 'totalcharges']]
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges.fillna(0) # should we set null values to 0?

# change yes no churn values to 0 and 1
df.churn = (df.churn == 'yes').astype(int)
print(df.head().T)


                                 0             1               2  \
customerid              7590-vhveg    5575-gnvde      3668-qpybk   
gender                      female          male            male   
seniorcitizen                    0             0               0   
partner                        yes            no              no   
dependents                      no            no              no   
tenure                           1            34               2   
phoneservice                    no           yes             yes   
multiplelines     no_phone_service            no              no   
internetservice                dsl           dsl             dsl   
onlinesecurity                  no           yes             yes   
onlinebackup                   yes            no             yes   
deviceprotection                no           yes              no   
techsupport                     no            no              no   
streamingtv                     no            no

In [22]:
#set up the validatin framework using scikit learn
#train_test_split only breaks the data into training and testing
# need extra step into train test validation
df_train_full, df_test = train_test_split(df, test_size = 0.2, random_state = 1) 
df_train, df_val = train_test_split(df_train_full, test_size = 0.25, random_state = 11)

#reset indexes for prettiness
df_train = df_train.reset_index(drop = True)
df_val = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']
# check split amounts
len(df_train), len(df_test), len(df_val), len(df_train) + len(df_test) + len(df_val), len(df)

(4225, 1409, 4225, 9859, 7043)

In [23]:
# exploring the data
#df_train_full.isnull().sum()
df_train_full.churn.value_counts()
global_mean = df_train_full.churn.mean()
round(global_mean, 2)

np.float64(0.27)

In [24]:
# list of string columns
#categorical = pd.concat([df_train, df_val]).select_dtypes(include='object').columns.tolist()
#numerical = pd.concat([df_train, df_val]).select_dtypes(include=['int64', 'float']).columns.tolist()

#manual to align with course and use columns that make sense
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

print (categorical)
print (numerical)

['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']
['tenure', 'monthlycharges', 'totalcharges']


In [25]:
# profile string datasets
#results = []
#for col in categorical:
#    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
#    df_group['diff'] = df_group['mean'] - global_mean
#    df_group['risk'] = df_group['mean'] / global_mean
#    df_group['variable'] = col
#    df_group = df_group.reset_index().rename(columns= {col:'category'})
#    results.append(df_group)
#    #display (df_group) #this shows individual records

 #combine all into one dataframe
#df_summary = pd.concat(results, ignore_index = True)
#df_summary[['variable', 'category', 'mean', 'diff', 'risk']]


for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


In [26]:
# churn rate
# percent of churn = 1
global_churn_rate = df_train_full.churn.mean()
# this also works
df_train_full.churn.value_counts(normalize = True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [27]:
# unique values in columns
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [28]:
#feature importance
# churn rate within group vs global churn rate. Do they over or under index?
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print ('gender==femaile:', round(female_mean, 3))

male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print ('gender == male:', round(male_mean, 3))

print ('female mean / global:', round(female_mean / global_mean, 3))
print ('male mean / global:', round(male_mean / global_mean, 3))

gender==femaile: 0.277
gender == male: 0.263
female mean / global: 1.025
male mean / global: 0.975


In [29]:
df_group = df_train_full.groupby(by = 'gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean']/ global_mean
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [30]:
global_mean

np.float64(0.26996805111821087)

In [31]:
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


In [35]:
def calculate_mi(series):
    return mutual_info_score(series, sf_train_full.churn)

    df_mi = df_train_full[categorical].apply(calcualte_mi)
    df_mi = df_mi.sort_values(ascending=False).to_frame(name='M1')

    display(df_mi.head())
    display(df_mi.tail())

#numeric correlations
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('correlation')
# churn results for numeric
df_train_full.groupby(by='churn')[numerical].mean()

,tenure,monthlycharges,totalcharges
churn,,,
0,37.531972,61.176477,2552.987320
1,18.070348,74.521203,1545.689415
